In [1]:
# dependancies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data# config

In [2]:
# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# config
# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
# variables
batch_size = 40
num_steps = 500
model_dir = "/tmp/tfmodels/mnist_tf_learn_7"
data_dir = "/tmp/MNIST_data"

In [5]:
# define input generator function
# example calls: generate_input_fn(DATA_SETS.train) or generate_input_fn(DATA_SETS.test)
def generate_input_fn(dataset, batch_size=batch_size):
    def _input_fn():
        # create placeholders for x and y
        # x = images
        # y = labels
        X = tf.constant(dataset.images)
        Y = tf.constant(dataset.labels, dtype=tf.int32)
        
        # grab the next batch
        image_batch, label_batch = tf.train.shuffle_batch([X,Y],
                               batch_size=batch_size,
                               capacity=8*batch_size,
                               min_after_dequeue=4*batch_size,
                               enqueue_many=True
                              )
        # return next batch
        return {'pixels': image_batch} , label_batch
    return _input_fn

In [6]:
# define and run the DNN classifier
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1, batch_size=batch_size):
    # define the feature_columns
    feature_columns = [tf.contrib.layers.real_valued_column("pixels", dimension=784)]

    # define the model
    # layer 1: 128 neurons
    # layer 2: 32 neurons
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=10,
        hidden_units=[128, 32],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
    )

    # fit the classifier to the dataset
    classifier.fit(input_fn=generate_input_fn(DATA_SETS.train, batch_size=batch_size),
                   steps=num_steps)

    print("Finished running the deep training via the fit() method")

    print("\n---Evaluating DNN classifier accuracy...")
    
    # get the accuracy of the classifier
    accuracy_score = classifier.evaluate(input_fn=generate_input_fn(DATA_SETS.test, batch_size=batch_size),
                                         steps=100)['accuracy']

    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

In [7]:
# define main function
def main(_):
    # read in data, downloading first if necessary
    global DATA_SETS
    print("Downloading and reading data sets...")
    DATA_SETS = input_data.read_data_sets(data_dir)

    print("\n---- Running DNN classifier...")
    # run the classifier
    define_and_run_dnn_classifier(num_steps, model_dir)

In [8]:
# run main function
if __name__ == '__main__':
    main(_)

Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz

---- Running DNN classifier...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f84b2c87518>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means tha

In [9]:
# Run the follow and go to localhost:6006 for data
# tensorboard --logdir /tmp/tfmodels/mnist_tf_learn_3